In [56]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog

def load_food_data(filepath):
    """ CSV 파일에서 음식 데이터를 불러오는 함수 """
    df = pd.read_csv(filepath)
    df["칼로리"] = df["칼로리"].astype(float)
    df["탄수화물"] = df["탄수화물"].astype(float)
    df["단백질"] = df["단백질"].astype(float)
    df["지방"] = df["지방"].astype(float)
    return df

def get_nutritional_targets(sex, body_type, goal):
    """ 사용자 입력에 따른 목표 칼로리 및 탄단지 비율 설정 """
    calorie_dict = {
        "사과형": {"diet": {"남성": 1500, "여성": 1300}, "muscle_gain": {"남성": 2500, "여성": 2200}},
        "배형": {"diet": {"남성": 1600, "여성": 1400}, "muscle_gain": {"남성": 2600, "여성": 2300}},
    }
    macro_ratios = {  # 탄:단:지 비율
        "diet": (50, 30, 20),
        "gain": (40, 40, 20),
        "maintain": (45, 35, 20)
    }
    total_calories = calorie_dict.get(body_type, {}).get(goal, {}).get(sex, 1800)
    carb_ratio, protein_ratio, fat_ratio = macro_ratios[goal]
    return total_calories, carb_ratio, protein_ratio, fat_ratio

def optimize_meal_plan(food_df, total_calories, macro_ratios):
    """ 최적화 문제를 풀어 탄단지 맞춘 식단 추천 """
    carb_target, protein_target, fat_target = [r / 100 * total_calories / 4 for r in macro_ratios]
    calorie_target = total_calories
    
    food_nutrients = food_df[["칼로리", "탄수화물", "단백질", "지방"]]
    num_foods = len(food_df)
    
    # 목적함수 계수 설정 (총 오차 최소화)
    c = np.ones(num_foods)
    
    # 제약 조건
    A_eq = np.array([
        food_nutrients["칼로리"].values,
        food_nutrients["탄수화물"].values,
        food_nutrients["단백질"].values,
        food_nutrients["지방"].values
    ])
    b_eq = np.array([calorie_target, carb_target, protein_target, fat_target])
    
    # 각 음식의 섭취량이 0 이상
    bounds = [(0, None)] * num_foods
    
    # 최적화 실행
    res = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method="highs")
    
    if res.success:
        selected_foods = food_df.iloc[np.where(res.x > 0.01)]
        return selected_foods
    else:
        return None
        

# 실행 예시
# file_path = "data/food_nutrition_data.csv"
# food_data = load_food_data(file_path)
# total_cal, c_ratio, p_ratio, f_ratio = get_nutritional_targets("남성", "사과형", "diet")
# food_data = food_data.fillna(0)  # NaN을 0으로 채우기
# recommended_meal = optimize_meal_plan(food_data, total_cal, (c_ratio, p_ratio, f_ratio))
# print(recommended_meal)


In [59]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog
import re

def get_user_input():
    """ 사용자로부터 성별, 체형, 목표 입력 받기 """
    gender = input("성별을 입력하세요 (남성/여성): ")
    body_type = input("체형을 입력하세요 (사과형/배형/모래시계형/엉덩이형/상체형/하체형/표준체형): ")
    goal = input("목표를 입력하세요 (diet/maintain/gain): ")
    return gender, body_type, goal

def preprocess_food_data(food_df, body_type):
    """ '1회 섭취량' 컬럼을 숫자로 변환하고 체형별 필터링 수행 """
    def convert_serving_size(value):
        match = re.search(r'\d+', str(value))
        return float(match.group()) if match else np.nan
    
    food_df["1회 섭취량"] = food_df["1회 섭취량"].apply(convert_serving_size)
    food_df.dropna(subset=["1회 섭취량"], inplace=True)
    
    # 체형별 추천 및 제한 음식 필터링
    body_type_filters = {
        "사과형": {"exclude": ["고탄수화물", "고지방 간식"], "include": ["고단백 음식", "저지방 음식"]},
        "배형": {"exclude": ["고당도 음식", "고탄수화물"], "include": ["저탄수화물 음식", "고섬유 음식"]},
        "모래시계형": {"exclude": [], "include": []},
        "엉덩이형": {"exclude": ["저단백 음식"], "include": ["고단백 음식"]},
        "상체형": {"exclude": ["고지방 음식"], "include": ["저지방 단백질"]},
        "하체형": {"exclude": ["고나트륨 음식"], "include": ["저나트륨 음식"]},
        "표준체형": {"exclude": [], "include": []}
    }
    
    filters = body_type_filters.get(body_type, {})
    exclude_categories = filters.get("exclude", [])
    include_categories = filters.get("include", [])
    
    food_df = food_df[~food_df["카테고리"].isin(exclude_categories)]
    if include_categories:
        food_df = food_df[food_df["카테고리"].isin(include_categories)]
    
    return food_df

def optimize_meal_plan(food_df, total_calories, macro_ratios):
    """
    총 칼로리 및 탄단지 비율을 고려하여 최적의 식단을 구성하는 함수 (최대 섭취량 제한 추가)
    """
    c_ratio, p_ratio, f_ratio = macro_ratios
    num_foods = len(food_df)
    
    # 목적 함수 (칼로리 최소화)
    c = food_df["칼로리"].values
    
    # 제약 조건 설정 (탄수화물, 단백질, 지방 비율 및 최대 섭취량 제한)
    A_eq = [
        food_df["탄수화물"].values,
        food_df["단백질"].values,
        food_df["지방"].values,
        food_df["칼로리"].values
    ]
    b_eq = [
        total_calories * (c_ratio / 100) / 4,
        total_calories * (p_ratio / 100) / 4,
        total_calories * (f_ratio / 100) / 9,
        total_calories
    ]
    
    # 각 음식 최소 및 최대 선택량 설정 (1회 섭취량 기반 제한)
    bounds = [(0, food_df.iloc[i]["1회 섭취량"] / 100) for i in range(num_foods)]
    
    # 최적화 실행
    res = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method="highs")
    
    if res.success:
        selected_foods = food_df.iloc[np.where(res.x > 0.01)]
        return selected_foods
    else:
        print("최적화 실패. 기본 식단을 반환합니다.")
        return food_df.sample(n=3)

def format_meal_plan(meal_df, total_calories, c_ratio, p_ratio, f_ratio):
    """
    추천된 식단을 아침, 점심, 저녁으로 구분하여 보기 좋게 출력하는 함수 (탄단지 고려)
    """
    meal_ratios = {"아침": 0.3, "점심": 0.4, "저녁": 0.3}
    meal_plan = {"아침": [], "점심": [], "저녁": []}
    meal_targets = {meal: total_calories * ratio for meal, ratio in meal_ratios.items()}
    
    for meal_type in ["아침", "점심", "저녁"]:
        remaining_calories = meal_targets[meal_type]
        remaining_carb = c_ratio / 100 * total_calories / 4 * meal_ratios[meal_type]
        remaining_protein = p_ratio / 100 * total_calories / 4 * meal_ratios[meal_type]
        remaining_fat = f_ratio / 100 * total_calories / 9 * meal_ratios[meal_type]
        
        shuffled_df = meal_df.sample(frac=1).reset_index(drop=True)
        for idx, row in shuffled_df.iterrows():
            if (row["칼로리"] <= remaining_calories and 
                row["탄수화물"] <= remaining_carb and
                row["단백질"] <= remaining_protein and
                row["지방"] <= remaining_fat and
                row["1회 섭취량"] <= 500):
                meal_plan[meal_type].append(row)
                remaining_calories -= row["칼로리"]
                remaining_carb -= row["탄수화물"]
                remaining_protein -= row["단백질"]
                remaining_fat -= row["지방"]
            if len(meal_plan[meal_type]) == 0:
                meal_plan[meal_type].append(shuffled_df.iloc[0])
    
      print("\n===== 추천 식단 =====")
    for meal, foods in meal_plan.items():
        print(f"\n{meal}:")
        for _, food in pd.DataFrame(foods).iterrows():
            print(f"- {food['식품명(한글)']} (1회 섭취량: {food['1회 섭취량']}g, 칼로리: {food['칼로리']}kcal, 탄: {food['탄수화물']}g, 단: {food['단백질']}g, 지: {food['지방']}g)")
    
    return meal_plan

# 사용자 입력 받기
gender, body_type, goal = get_user_input()

# 예시: 사용자 입력을 이용해 최적화 함수 실행 (최적화 함수는 별도로 구현 필요)
# food_data = preprocess_food_data(food_data)  # 데이터 전처리 추가
total_cal, c_ratio, p_ratio, f_ratio = get_nutritional_targets(gender, body_type, goal)
recommended_meal = optimize_meal_plan(food_data, total_cal, (c_ratio, p_ratio, f_ratio))
format_meal_plan(recommended_meal, total_cal, c_ratio, p_ratio, f_ratio)


성별을 입력하세요 (남성/여성):  여성
체형을 입력하세요 (사과형/배형/모래시계형/엉덩이형/상체형/하체형/표준체형):  모래시계형
목표를 입력하세요 (diet/maintain/gain):  maintain



===== 추천 식단 =====

아침:
- 데리야끼치킨 (1회 섭취량: 100.0g, 칼로리: 1243.0kcal, 탄: 56.96g, 단: 166.04g, 지: 32.81g)
- 고추장 (1회 섭취량: 100.0g, 칼로리: 80.0kcal, 탄: 18.28g, 단: 3.88g, 지: 0.64g)
- 랍스타 (1회 섭취량: 100.0g, 칼로리: 98.0kcal, 탄: 1.28g, 단: 20.5g, 지: 0.59g)
- 가지 (1회 섭취량: 100.0g, 칼로리: 24.0kcal, 탄: 5.7g, 단: 1.01g, 지: 0.19g)
- 파슬리 (1회 섭취량: 100.0g, 칼로리: 36.0kcal, 탄: 6.33g, 단: 2.97g, 지: 0.79g)
- 오리구이 (1회 섭취량: 100.0g, 칼로리: 132.0kcal, 탄: 0.0g, 단: 18.28g, 지: 5.95g)

점심:
- 랍스타 (1회 섭취량: 100.0g, 칼로리: 98.0kcal, 탄: 1.28g, 단: 20.5g, 지: 0.59g)
- 파슬리 (1회 섭취량: 100.0g, 칼로리: 36.0kcal, 탄: 6.33g, 단: 2.97g, 지: 0.79g)
- 고추장 (1회 섭취량: 100.0g, 칼로리: 80.0kcal, 탄: 18.28g, 단: 3.88g, 지: 0.64g)
- 치킨스테이크 (1회 섭취량: 100.0g, 칼로리: 130.0kcal, 탄: 0.0g, 단: 24.0g, 지: 1.5g)
- 콩나물 (1회 섭취량: 100.0g, 칼로리: 30.0kcal, 탄: 5.94g, 단: 3.04g, 지: 0.18g)
- 가지 (1회 섭취량: 100.0g, 칼로리: 24.0kcal, 탄: 5.7g, 단: 1.01g, 지: 0.19g)
- 케일 (1회 섭취량: 100.0g, 칼로리: 50.0kcal, 탄: 10.01g, 단: 3.3g, 지: 0.7g)
- 밀감 (1회 섭취량: 100.0g, 칼로리: 31.0kcal, 탄: 5.13g, 단: 2.57g, 지: 0.82g)

저녁:
- 데리야끼

{'아침': [식품명(한글)    데리야끼치킨
  칼로리        1243.0
  탄수화물        56.96
  단백질        166.04
  지방          32.81
  1회 섭취량      100.0
  Name: 0, dtype: object,
  식품명(한글)      고추장
  칼로리         80.0
  탄수화물       18.28
  단백질         3.88
  지방          0.64
  1회 섭취량     100.0
  Name: 2, dtype: object,
  식품명(한글)      랍스타
  칼로리         98.0
  탄수화물        1.28
  단백질         20.5
  지방          0.59
  1회 섭취량     100.0
  Name: 3, dtype: object,
  식품명(한글)       가지
  칼로리         24.0
  탄수화물         5.7
  단백질         1.01
  지방          0.19
  1회 섭취량     100.0
  Name: 4, dtype: object,
  식품명(한글)      파슬리
  칼로리         36.0
  탄수화물        6.33
  단백질         2.97
  지방          0.79
  1회 섭취량     100.0
  Name: 5, dtype: object,
  식품명(한글)     오리구이
  칼로리        132.0
  탄수화물         0.0
  단백질        18.28
  지방          5.95
  1회 섭취량     100.0
  Name: 6, dtype: object],
 '점심': [식품명(한글)      랍스타
  칼로리         98.0
  탄수화물        1.28
  단백질         20.5
  지방          0.59
  1회 섭취량     100.0
  Name: 0, dtype: object,
 

In [82]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error

# 데이터 불러오기
file_path = "data/generated_meal_plan.csv"
df = pd.read_csv(file_path)

# 필요 없는 열 제거 (아침, 점심, 저녁 음식명은 학습에 사용하지 않음)
df = df.drop(columns=["아침", "점심", "저녁"])

# 범주형 변수 변환 (성별, 체형, 목표 → 숫자로 인코딩)
label_encoders = {}
for col in ["성별", "체형", "목표"]:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # 인코더 저장 (추후 예측할 때 필요)

# 입력(X)과 출력(y) 분리
X = df[["성별", "체형", "목표"]]
y = df[["총 칼로리", "탄수화물", "단백질", "지방"]]

# 데이터 분할 (학습용 80%, 검증용 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.fillna(X_train.mean())  # 평균값으로 대체
X_test = X_test.fillna(X_test.mean())
y_train = y_train.fillna(y_train.mean())
y_test = y_test.fillna(y_test.mean())

# XGBoost 모델 학습
model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=300, learning_rate=0.01, max_depth=3, subsample=0.8)
model.fit(X_train, y_train)

# 예측 수행
y_pred = model.predict(X_test)

# 평가 (평균 절대 오차)
mae = mean_absolute_error(y_test, y_pred)
print(f"✅ 모델 학습 완료! 평균 절대 오차 (MAE): {mae:.2f}")

# 모델 저장
model.save_model("data/xgboost_meal_plan.json")
print("✅ 모델 저장 완료! 파일: xgboost_meal_plan.json")


✅ 모델 학습 완료! 평균 절대 오차 (MAE): 240.83
✅ 모델 저장 완료! 파일: xgboost_meal_plan.json


In [14]:
from sklearn.preprocessing import LabelEncoder

# 변환할 컬럼
label_cols = ["성별", "체형", "목표"]

# 각 컬럼에 대해 LabelEncoder 적용
label_encoders = {}
for col in label_cols:
    le = LabelEncoder()
    food_data[col] = le.fit_transform(food_data[col])
    label_encoders[col] = le  # 나중에 복원할 수 있도록 저장

print(food_data)

    성별  체형  목표        아침        점심        저녁   총 칼로리    탄수화물     단백질      지방
0    0   2   0       라자냐   브로컬리계란찜      브라우니  1847.0  271.53   37.47   74.30
1    0   2   2      연어초밥      야채볶음      당근주스     NaN     NaN     NaN     NaN
2    0   2   1         잣       오미자     닭고기볶음   567.0   32.00   36.80   33.79
3    0   1   0      핫케이크    함박스테이크         귤   515.0   39.30   34.53   24.92
4    0   1   2       파파야        게장       조미김   156.0   12.81   20.81    1.92
5    0   1   1   베이글샌드위치      연어초밥      스파게티   866.0   84.63   55.90   32.38
6    0   0   0    페퍼로니피자      땅콩버터      훈제오리  1201.0   51.02   56.41   89.97
7    0   0   2    참치샌드위치       크래커      아이스티   621.0   61.95   32.94   26.12
8    0   0   1        도넛        배추       팟타이  2662.0  186.45  132.29  152.90
9    0   4   0       쉐이크      사과주스     쇠고기볶음   527.0   38.81   29.25   29.01
10   0   4   2      스파게티        인삼       짜장면   828.0  134.86   19.80   22.93
11   0   4   1        와인      미소라멘        잡채  1380.0   82.63   88.45   69.42

In [20]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

# (1) 데이터 로드
food_data = pd.read_csv("data/generated_meal_plan.csv")  # 네 CSV 파일 경로를 사용해야 함

# 변환할 컬럼
label_cols = ["성별", "체형", "목표", "아침", "점심", "저녁", "탄수화물", "단백질", "지방"]

# 각 컬럼에 대해 LabelEncoder 적용
label_encoders = {}
for col in label_cols:
    le = LabelEncoder()
    food_data[col] = le.fit_transform(food_data[col])
    label_encoders[col] = le  # 나중에 복원할 수 있도록 저장

food_data.fillna(0, inplace=True)

# (2) 입력 변수(X)와 타겟(Y) 설정
X = food_data.drop(columns=["총 칼로리"])  # 예측할 값 제외
y = food_data["총 칼로리"]  # 예측할 타겟

# (3) 데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# (4) 개별 모델 정의
base_models = [
    ("xgb", XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)),
    ("lgbm", LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=42)),
    ("rf", RandomForestRegressor(n_estimators=100, random_state=42))
]

# (5) 스태킹 모델 생성 (메타 모델로 Ridge 사용)
stacking_model = StackingRegressor(estimators=base_models, final_estimator=Ridge())

# (6) 학습 및 평가
stacking_model.fit(X_train, y_train)
score = stacking_model.score(X_test, y_test)
print(f"스태킹 모델 성능: {score:.4f}")


C:\Users\fn45\anaconda3\envs\myenv\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] 지정된 파일을 찾을 수 없습니다
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\fn45\anaconda3\envs\myenv\lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
  File "C:\Users\fn45\anaconda3\envs\myenv\lib\subprocess.py", line 493, in run
    with Popen(*popenargs, **kwargs) as process:
  File "C:\Users\fn45\anaconda3\envs\myenv\lib\subprocess.py", line 858, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\fn45\anaconda3\envs\myenv\lib\subprocess.py", line 1327, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 33, number of used features: 0
[LightGBM] [Info] Start training from score 826.939394
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the sp